In [20]:
# This will prompt you to enter your API key if needed and it will save it to a config file.
# Note that API keys should be kept secret like passwords.
sm.save_login()

Please generate an API key at https://metaspace2020.eu/user/me and enter it here (or leave blank to cancel): ········


Saved API key to /Users/christoffer/.metaspace


In [39]:
datasets = pd.read_csv('/Users/christoffer/Downloads/metaspace_datasets (1).csv')

In [40]:
datasets.datasetId

,datasetId,datasetName,group,submitter,PI,organism,organismPart,condition,growthConditions,ionisationSource,maldiMatrix,analyzer,resPower400,polarity,uploadDateTime,FDR@10%,database,opticalImage
0,2023-07-26_23h31m14s,C57BL/6_A349_NEG,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,Mus musculus (mouse),Brain,PFA fixed,NaN,AP-MALDI,"1,5-diaminonaphthalene (DAN)",Exploris 480,240000,negative,2023-07-26T23:31:14.204085,134,LipidMaps - 2017-12-12,No optical image
1,2023-07-27_04h55m15s,C57BL/6_C369_NEG,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,Mus musculus (mouse),Brain,PFA fixed,NaN,AP-MALDI,"1,5-diaminonaphthalene (DAN)",Exploris 480,240000,negative,2023-07-27T02:55:15.261000,145,LipidMaps - 2017-12-12,No optical image
2,2023-07-27_04h52m28s,C57BL/6_C366_NEG,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,Mus musculus (mouse),Brain,PFA fixed,NaN,AP-MALDI,"1,5-diaminonaphthalene (DAN)",Exploris 480,240000,negative,2023-07-27T04:52:28.840581,133,LipidMaps - 2017-12-12,No optical image
3,2023-07-27_04h54m17s,C57BL/6_C371_NEG,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,Mus musculus (mouse),Brain,PFA fixed,NaN,AP-MALDI,"1,5-diaminonaphthalene (DAN)",Exploris 480,240000,negative,2023-07-27T02:54:17.275000,143,LipidMaps - 2017-12-12,No optical image
4,2023-07-14_16h57m39s,C57BL/6_A348_NEG,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,Mus musculus (mouse),Brain,PFA fixed,NaN,AP-MALDI,"1,5-diaminonaphthalene (DAN)",Exploris 480,240000,positive,2023-07-14T16:57:39.553673,82,LipidMaps - 2017-12-12,No optical image
5,2023-07-10_03h21m10s,C57BL/6_B361_NEG,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,Mus musculus (mouse),Brain,PFA fixed,NaN,AP-MALDI,"1,5-diaminonaphthalene (DAN)",Exploris 480,240000,positive,2023-07-10T03:21:10.148472,84,LipidMaps - 2017-12-12,No optical image
6,2023-07-13_18h23m19s,C57BL/6_B357_NEG,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,Mus musculus (mouse),Brain,PFA fixed,NaN,AP-MALDI,"1,5-diaminonaphthalene (DAN)",Exploris 480,240000,positive,2023-07-13T18:23:19.868979,84,LipidMaps - 2017-12-12,No optical image
7,2023-07-14_16h56m54s,C57BL/6_A354_NEG,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,Mus musculus (mouse),Brain,PFA fixed,NaN,AP-MALDI,"1,5-diaminonaphthalene (DAN)",Exploris 480,240000,positive,2023-07-14T16:56:54.669660,78,LipidMaps - 2017-12-12,No optical image
8,2023-05-26_00h33m42s,C57BL/6_B358_NEG,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,Mus musculus (mouse),Brain,PFA fixed,NaN,AP-MALDI,"1,5-diaminonaphthalene (DAN)",Exploris 480,240000,negative,2023-05-26T00:33:42.512173,111,SwissLipids - 2018-02-02,No optical image
9,2023-05-23_06h43m54s,C57BL/6_C369,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,Mus musculus (mouse),Brain,Free-floating,NaN,AP-MALDI,alpha-cyano-4-hydroxycinnamic acid (CHCA),Exploris 480,240000,positive,2023-05-23T02:43:54.078000,123,SwissLipids - 2018-02-02,No optical image


In [49]:
adata_ = []
for dataset in datasets.datasetId.unique():
    print(dataset)
    
    adata = metaspace_to_anndata(
        dataset_id=dataset,
        fdr=0.1,
        #database=("BraChemDB", "2018-01"),
    )
    adata.obs['sample_id'] = dataset

    adata_.append(adata)

2023-07-26_23h31m14s


100%|████████████████████████████████████████| 103/103 [00:01<00:00, 74.67it/s] 


2023-07-27_04h55m15s


100%|████████████████████████████████████████| 126/126 [00:01<00:00, 63.02it/s] 


2023-07-27_04h52m28s


...

100%|████████████████████████████████████████| 21/21 [00:00<00:00, 35.13it/s]   


2023-05-07_16h51m58s


100%|████████████████████████████████████████| 34/34 [00:00<00:00, 47.95it/s]   


2023-05-07_16h45m33s


100%|████████████████████████████████████████| 19/19 [00:00<00:00, 34.78it/s]   


In [43]:
adata = sc.concat(
    adata_,
    axis=0,
    join="outer"  # 'inner' to keep only shared vars
)

/Users/christoffer/miniconda3/envs/sc_py312/lib/python3.12/site-packages/anndata/_core/anndata.py:1791: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [50]:
for column in datasets.columns:
    adata.obs[column] = adata.obs.sample_id.map(dict(zip(datasets['datasetId'], datasets[column])))

In [52]:
adata.obs

,ion_image_pixel_x,ion_image_pixel_y,ion_image_shape_y,ion_image_shape_x,sample_id,datasetId,datasetName,group,submitter,PI,...,growthConditions,ionisationSource,maldiMatrix,analyzer,resPower400,polarity,uploadDateTime,FDR@10%,database,opticalImage
ion_image_pixel,,,,,,,,,,,,,,,,,,,,,
0,0,0,350,330,2023-05-07_16h45m33s,2023-05-07_16h45m33s,C57BL/6_C366,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,...,NaN,AP-MALDI,alpha-cyano-4-hydroxycinnamic acid (CHCA),Exploris 480,240000,positive,2023-05-07T16:45:33.641604,175,SwissLipids - 2018-02-02,No optical image
1,1,0,350,330,2023-05-07_16h45m33s,2023-05-07_16h45m33s,C57BL/6_C366,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,...,NaN,AP-MALDI,alpha-cyano-4-hydroxycinnamic acid (CHCA),Exploris 480,240000,positive,2023-05-07T16:45:33.641604,175,SwissLipids - 2018-02-02,No optical image
2,2,0,350,330,2023-05-07_16h45m33s,2023-05-07_16h45m33s,C57BL/6_C366,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,...,NaN,AP-MALDI,alpha-cyano-4-hydroxycinnamic acid (CHCA),Exploris 480,240000,positive,2023-05-07T16:45:33.641604,175,SwissLipids - 2018-02-02,No optical image
3,3,0,350,330,2023-05-07_16h45m33s,2023-05-07_16h45m33s,C57BL/6_C366,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,...,NaN,AP-MALDI,alpha-cyano-4-hydroxycinnamic acid (CHCA),Exploris 480,240000,positive,2023-05-07T16:45:33.641604,175,SwissLipids - 2018-02-02,No optical image
4,4,0,350,330,2023-05-07_16h45m33s,2023-05-07_16h45m33s,C57BL/6_C366,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,...,NaN,AP-MALDI,alpha-cyano-4-hydroxycinnamic acid (CHCA),Exploris 480,240000,positive,2023-05-07T16:45:33.641604,175,SwissLipids - 2018-02-02,No optical image
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115495,325,349,350,330,2023-05-07_16h45m33s,2023-05-07_16h45m33s,C57BL/6_C366,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,...,NaN,AP-MALDI,alpha-cyano-4-hydroxycinnamic acid (CHCA),Exploris 480,240000,positive,2023-05-07T16:45:33.641604,175,SwissLipids - 2018-02-02,No optical image
115496,326,349,350,330,2023-05-07_16h45m33s,2023-05-07_16h45m33s,C57BL/6_C366,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,...,NaN,AP-MALDI,alpha-cyano-4-hydroxycinnamic acid (CHCA),Exploris 480,240000,positive,2023-05-07T16:45:33.641604,175,SwissLipids - 2018-02-02,No optical image
115497,327,349,350,330,2023-05-07_16h45m33s,2023-05-07_16h45m33s,C57BL/6_C366,NaN,Krzysztof Urbanowicz,Krzysztof Urbanowicz,...,NaN,AP-MALDI,alpha-cyano-4-hydroxycinnamic acid (CHCA),Exploris 480,240000,positive,2023-05-07T16:45:33.641604,175,SwissLipids - 2018-02-02,No optical image
